In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


In [2]:
print("langchain advanced")

langchain advanced


In [3]:
#bedrock client 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # replace with your region
)

In [4]:
#llm
llm = ChatBedrockConverse(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    temperature=0.7,
    max_tokens=2000
)

In [50]:
async def extract_stream(chain, input_json):
    collected_text = []
    async for event in chain.astream_events(input_json):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        print(text, end="", flush=True)
    return "".join(collected_text)


In [5]:
#input { json 포맷 }  -> prompt(text로 변환) -> llm(bedrock) -> output(string타입, json타입)
#여기서 input { json 포맷 }을 활용하는 방법에 대해 이해해보자 ( 여행에 관한 정보를 얻는 llm를 만든다고 가정한다 )

In [6]:
prompt_capital = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 수도에 대해 알려줘")
])

capital_chain = prompt_capital | llm |  StrOutputParser()


capital_chain.invoke({"country" : "태국"})

'태국의 수도는 방콕(Bangkok)입니다.\n\n방콕은 태국 중부에 위치한 대도시로, 약 800년 전부터 이 지역에 도시가 형성되기 시작했습니다. 현재 인구는 약 800만 명 정도로 태국 최대 도시입니다.\n\n주요 명소로는 왕궁과 사원들이 유명합니다.\n- 왕궁(그랜드 팰리스) - 태국 왕실의 궁전\n- 왓포(Wat Pho) - 거대한 황금 불상이 있는 사원\n- 왓아런(Wat Arun) - 석조 사원으로 "새벽의 사원"이라고도 불림\n\n또한 활기찬 시장과 고대 운하, 현대적인 쇼핑몰 등 볼거리가 많습니다. 치앙mai 강을 중심으로 수상 시장도 유명합니다. 태국 전통 문화와 현대 문화가 공존하는 활기찬 도시입니다.'

In [7]:
prompt_culture = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 문화에 대해 알려줘")
])

culture_chain = prompt_culture | llm |  StrOutputParser()


culture_chain.invoke({"country" : "태국"})


'태국은 아름다운 문화유산과 다양한 전통을 가지고 있는 나라입니다. 태국 문화의 주요 특징 몇 가지를 소개해드리겠습니다.\n\n1. 불교 문화\n태국은 불교 국가로, 태국인의 95% 이상이 불교를 믿고 있습니다. 태국 문화 전반에 불교 사상과 가치관이 스며들어 있습니다.\n\n2. 왕실 문화\n태국에는 오랜 역사를 가진 왕실이 있으며, 국민들은 왕실을 존경하고 있습니다. 왕실 의식과 행사는 태국 문화의 중요한 부분입니다.\n\n3. 전통 공예\n태국에는 독특한 전통 공예품들이 있습니다. 실크 직조, 목각, 은세공, 도자기 등 수준 높은 공예 기술이 발달해 있습니다.\n\n4. 전통 무용\n태국 전통 무용인 크론무용은 화려하고 우아한 동작으로 유명합니다. 왕실 의식이나 축제 때 공연됩니다.\n\n5. 태국 요리\n태국 요리는 세계적으로 유명하며, 향신료를 많이 사용하고 맵고 자극적인 맛이 특징입니다. 패드타이, 톰얌꿍 등이 대표적입니다.\n\n이외에도 태국에는 다양한 축제, 건축물, 예술 등 풍부한 문화유산이 있습니다. 전통과 현대가 공존하는 매력적인 문화를 가지고 있습니다.'

In [8]:
prompt_climate = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 날씨에 대해 알려줘")
])

climate_chain = prompt_climate | llm |  StrOutputParser()


climate_chain.invoke({"country" : "태국"})

'태국은 열대 몬순 기후를 가지고 있어 연중 고온다습한 날씨가 지속됩니다. 구체적인 날씨 특징은 다음과 같습니다:\n\n1. 기온\n- 연평균 기온은 약 27-28°C 정도입니다.\n- 최저 기온은 12-16°C, 최고 기온은 33-35°C 정도입니다.\n- 4월-5월이 가장 더운 시기이고, 12월-1월이 가장 서늘한 시기입니다.\n\n2. 강수량\n- 연평균 강수량은 약 1,200-1,600mm 정도입니다. \n- 5월-10월이 우기로 많은 비가 내립니다.\n- 11월-4월이 건기로 비교적 건조합니다.\n\n3. 몬순\n- 5월-10월은 남서 몬순, 11월-2월은 북동 몬순의 영향을 받습니다.\n- 몬순 기간에는 폭우와 열대성 저기압의 영향을 받기도 합니다.\n\n4. 습도\n- 연중 높은 습도를 보이며, 특히 우기에는 80% 이상의 높은 습도가 유지됩니다.\n\n태국은 열대 기후의 영향으로 연중 덥고 습한 날씨가 지속되지만, 지역과 계절에 따라 다소 차이가 있습니다.'

In [9]:
prompt_location = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 유명한 관광 명소에 대해 알려줘")
])

climate_location = prompt_location | llm |  StrOutputParser()


climate_location.invoke({"country" : "태국"})

"태국은 아름다운 자연경관과 유서깊은 문화유적으로 유명한 관광지입니다. 주요 관광명소는 다음과 같습니다:\n\n1. 방콕 - 왓포 사원, 왕궁, 카오산로드 등\n\n2. 아유타야 - 유네스코 세계문화유산인 아유타야 유적지\n\n3. 치앙마이 - 도이수텝 사원, 밤부 야시장, 동굴사원 등 \n\n4. 푸켓 - 아름다운 해변과 열대 리조트\n\n5. 파타야 - 해변과 나이트라이프가 유명\n\n6. 코사무이 - 에메랄드 빛 바다와 아름다운 해변\n\n7. 피피섬 - 영화 '비치'의 촬영지로 유명한 섬\n\n8. 방콕 인근 amphawa 부나콘 등 운하마을\n\n태국에는 불교문화 유적지, 아름다운 해변과 섬, 활기찬 도시와 마을 등 다양한 관광자원이 있습니다. 현지 음식과 마사지도 태국 여행의 즐거움입니다."

In [10]:
#여행에 관한 정보를 얻는 llm를 만든다고 가정한다 위의 네가지 정보를 모두 포함하여 전체적인 형태의 여행 정보를 얻고 싶다고 가정 할때 어떻게 langchain에 활용할수 있을까?
#결국 llm에서 결과가 나오면 string임으로 이 string을 json안에 assgin가능하다

In [11]:
# input에 {"country": "태국"}이라고 두고 itemgetter("country")를 통해 변수를 가져온뒤 각각 llm의 결과를 얻고 sting으로 json에 포함시킨다면? 
# llm을 통과한 결과값들이 새로운 input(json타입)을 형성할수 있지 않을까?
new_input = {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | climate_chain
            }

In [12]:
# new_input의 capital, culture, climate, location가  template_total의 capital, culture, climate, location 변수와 연결된다
template_total = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 가기 가장 좋은 계절을 알려주세요
"""
prompt_total  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_total)

In [13]:
travel_chain = new_input | prompt_total | llm | StrOutputParser()

In [14]:
travel_chain.invoke({"country" : "태국"})

'태국을 여행하기에 가장 좋은 시기는 건기인 11월에서 4월 사이가 될 것 같습니다. 이 시기의 장점은 다음과 같습니다:\n\n1. 온화한 기온\n건기 동안 태국의 기온은 연중 가장 서늘한 편입니다. 특히 12월부터 2월 사이가 가장 선선해서 밤낮으로 지나치게 덥지 않습니다.\n\n2. 건조한 날씨\n이 시기에는 북동무역풍의 영향으로 상대적으로 건조한 날씨가 유지됩니다. 우기에 비해 습도도 낮아 더위를 견디기 쉽습니다.\n\n3. 적은 강수량\n우기인 5월에서 10월에 비해 강수량이 적어 관광하기에 좋습니다. 홍수 등의 자연재해 위험도 낮습니다.\n\n4. 축제 즐기기 좋음\n송크란(태국 새해) 축제가 4월에 열리는데, 건기 날씨 덕분에 축제를 더욱 즐길 수 있습니다.\n\n다만 건기 중에서도 3~4월은 가장 더운 시기이므로 피하는 것이 좋겠습니다. 전반적으로 11월에서 2월 사이가 태국 여행에 가장 적절한 시기가 될 것 같습니다.'

In [16]:
#비슷한 방법으로 RunnableParallel도 존재한다 
parallel_chain = RunnableParallel(
                                  capital=capital_chain, 
                                  culture=culture_chain,
                                  climate=climate_chain, 
                                  location = climate_location
                                 )

In [17]:
travel_chain_parallel =  parallel_chain | prompt_total | llm | StrOutputParser()

In [18]:
travel_chain_parallel.invoke({"country" : "태국"})

'태국을 여행하기에 가장 좋은 시기는 일반적으로 건기인 11월부터 4월까지입니다. 이 시기의 장점은 다음과 같습니다:\n\n1. 맑고 건조한 날씨\n- 이 기간 동안 태국 전역에 걸쳐 맑은 날씨가 지속되어 여행하기에 좋습니다. \n- 소나기 등의 비가 거의 오지 않아 야외 활동을 즐기기 적합합니다.\n\n2. 선선한 기온\n- 낮 기온은 30도 전후, 밤 기온은 20도 내외로 덥지 않고 여행하기에 쾌적합니다.\n- 4월 중순이 가장 덥지만, 다른 달에 비해 그렇게 높지는 않습니다.\n\n3. 축제 시기\n- 송크란(태국 새해) 축제가 4월에 열려 태국의 문화를 체험할 수 있습니다.\n- 11월에는 로이크라통(등불축제)이 있어 아름다운 등불 행렬을 볼 수 있습니다.\n\n다만, 건기에는 관광객이 몰리는 성수기라 숙박료가 비싸고 관광지가 혼잡할 수 있습니다. 그래도 전반적으로 날씨와 행사 등을 고려하면 11월부터 4월 사이가 태국 여행에 가장 적합한 시기라고 할 수 있겠습니다.'

In [19]:
# llm의 outpu의 경우 StrOutputParser()를 활용해서 string으로 나오지만 llm.with_structured_output(obj)을 통해 json포맷으로 받을수 있다 
# llm의 결과가 json포맷으로 오면 이를 활용하여 다른 langchain에 활용할수 있다

In [22]:
template_plan = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_plan)

In [23]:
#llm의 결과값 포맷의 schema
class Travel(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    location : str = Field(description="국가 관광명소에 대한 설명")
    plan: str = Field(description="여행계획")


In [28]:
one_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | climate_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
)
 

In [29]:
one_chain.invoke({"country" : "태국"})

Travel(capital='방콕은 태국의 수도이자 최대 도시입니다. 차오프라야 강가에 위치한 방콕은 운하도시로도 유명하며, 왕궁과 사원 등 역사유적지와 현대 고층빌딩들이 공존하는 활기찬 대도시입니다.', culture='태국은 불교 문화가 삶에 깊이 배여있으며, 왕실 문화, 무예, 음식, 축제 등 다양한 전통문화 유산을 간직하고 있습니다. 오랜 역사와 독특한 문화가 현대와 조화를 이루고 있습니다.', climate='태국은 열대 몬순 기후로 연중 고온다습한 날씨가 지속됩니다. 건기(11월~4월)와 우기(5월~10월)로 구분되며, 우기 때는 폭우와 홍수에 주의해야 합니다. 방콕 등 중부지역은 연중 더운 날씨이지만 우기 때 홍수 피해가 있습니다.', location='태국은 동남아시아 인도차이나 반도 중앙부에 위치한 나라입니다. 북부는 산악지대, 중부는 평원지대, 남부는 열대 해안지역으로 이루어져 있습니다. 주요 강은 차오프라야 강이며, 아열대 기후와 열대 기후가 공존합니다.', plan='태국 여행을 계획한다면 11월~2월 사이의 건기가 가장 좋겠습니다. 이 시기에는 폭우 염려 없이 왕궁, 사원 등 실외 관광지를 둘러볼 수 있습니다. 또한 송크란 축제(4월 중순)나 등불축제(11월 초) 기간에 여행하면 태국의 독특한 문화를 체험할 수 있습니다. 방콕뿐 아니라 치앙마이, 푸켓 등 주요 도시와 해변 휴양지를 여행코스에 포함하면 좋습니다.')

In [44]:
#data는 llm의 결과 값을 의미합니다  RunnableLambda함수를 활용하면 data와 llm결과를 연결 시켜줄수 있습니다
def transform_keys(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "location": data.location, 
        "plan":     data.plan
    }

In [45]:
template_detail = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail  = PromptTemplate(input_variables=["capital", "culture", "climate", "location", "plan"],
                                 template=template_detail)

In [46]:
# RunnableLambda(transform_keys)는 llm과 prompt_detail의 데이터를 매핑 시켜주는 역할
chain_and_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | climate_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
    | RunnableLambda(transform_keys)
    | prompt_detail
    | llm
    | StrOutputParser()
)


In [54]:
chain_and_chain.invoke({"country" : "일본"})

'물론입니다. 일본 여행 시 방문할 만한 주요 관광지와 축제에 대해 더 자세히 설명드리겠습니다.\n\n도쿄:\n- 긴자 거리 - 유명 쇼핑가로 전통과 현대가 공존하는 분위기\n- 아사쿠사 신사 - 도쿄에서 가장 오래된 불교 사원으로 호즈모ン 게이트가 유명\n- 도쿄 스카이트리 - 세계에서 가장 높은 나무 구조물인 방송탑\n\n교토: \n- 기온 전통가옥 지구 - 예로부터 귀족과 상류층이 거주했던 역사 지구\n- 금각사 - 화려한 금박으로 꾸며진 세계문화유산 사원\n- 아라시야마 - 협곡 사이 아름다운 대나무 숲이 있는 관광명소\n\n나라:\n- 동대사 - 세계 최대 규모의 청동 부처상이 있는 사원  \n- 나라 궁전 유적 - 8세기 헤이조쿄 시대 왕궁터 유적지\n\n히로시마:\n- 원폭 돔 - 세계평화기념물로 지정된 피폭 유적\n- 미요지마 - 일몰이 아름다운 에노시마 섬과 대교로 유명\n\n오키나와:  \n- 시코쿠 무토 해변 - 에메랄드빛 바다와 백사장이 유명  \n- 슈리성 - 옛 류큐왕국의 성곽 유적지\n\n축제:\n- 삿포로 눈축제 (2월) - 거대한 눈조각품을 감상하는 겨울축제\n- 기온 축제 (7월) - 전통의상 행렬, 불꽃놀이 등 교토 대표 축제\n- 나라 덴리 축제 (7-8월) - 일본 3대 축제 중 하나로 야간 행렬 볼거리\n\n이렇게 도시와 지역별 대표 관광지와 주요 축제에 대해 보다 구체적으로 설명드렸습니다. 일본 여행 계획에 도움이 되길 바랍니다.'

In [52]:
#사용했던 모든 데이터 출력
# result=await extract_stream(chain_and_chain, {"country" : "태국"})